# Dataset preparation and feature extraction

Here we provide code for dataset preparation and feature extraction in our paper. Ensure that the image data is placed in the folder `./image_data`, where the bright-field image at 0h (before CHIR treatment), the cell-containing regions, and the final cTnT fluorescence images for each well should be stored at `./image_data/CD00-*/[brightfield|cell_region|ctnt]/S*.png`. The information abour cell lines should be stored in the csv file `./image_data/cell_line_info.csv`. 

After running the notebook (which may take a long time), the data frames (`./dataset/CD00-*.pkl`) for later analysis will be obtained.

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import cv2
import tqdm
import random
from sklearn.model_selection import train_test_split

## Create Data frames

In [ ]:
image_data_dir = "./image_data/" # you may modify it if you save the image to somewhere else

batch_names = ["CD00-%d" % i for i in range(1, 11)]

cell_line_info = pd.read_csv(os.path.join(image_data_dir, "cell_line_info.csv"), index_col=0)
cell_line_info = dict(zip(cell_line_info.batch_name, cell_line_info.cell_line))

In [ ]:
os.makedirs("./dataset", exist_ok = True)

for batch_name in batch_names:
    
    df = []
    cell_line = cell_line_info[batch_name]
    S_ids = [int(os.path.split(img_name)[1][1:4]) for img_name in glob.glob(os.path.join(image_data_dir, batch_name,  "brightfield/*.png"))]
    
    for S_id in S_ids:
        df.append((batch_name, cell_line, S_id))
        
    df = pd.DataFrame(df, columns=["batch_name", "cell_line", "S_id"])
    df.to_pickle("./dataset/%s.pkl" % batch_name)
    df.to_csv("./dataset/%s.csv" % batch_name)

## Quantify the differentiation efficiency

Now we use the cTnT images to compute the Differentiation Efficiency Indexes. 

In [ ]:
for batch_name in batch_names:
    
    df = pd.read_pickle("./dataset/%s.pkl" % batch_name)
    
    print("Computing ", batch_name)
    
    for i in tqdm.tqdm(range(len(df))):
        
        ctnt_dir = os.path.join(image_data_dir, batch_name, "ctnt", "S%03d.png" % df.S_id[i])
        ctnt = cv2.imread(ctnt_dir, cv2.IMREAD_GRAYSCALE) / 255
        
        width0, width1 = ctnt.shape
        differentiation_efficiency_index = ctnt[ctnt > 0.5].sum() / (width0 * width1)
        
        df.at[i, "differentiation_efficiency_index"] = differentiation_efficiency_index
    
    df.to_pickle("./dataset/%s.pkl" % batch_name)
    df.to_csv("./dataset/%s.csv" % batch_name)

Compute the normalized differentiation efficiency for each well. Since the differentiation efficiency vary among batches, the normalization is performed on each cell lines.

In [ ]:
df_dict = dict()
for batch_name in batch_names:
    df = pd.read_pickle("./dataset/%s.pkl" % batch_name)
    df_dict[batch_name] = df
    
max_eff = dict([(cell_line, 0) for cell_line in cell_line_info.values()])
for batch_name in batch_names:
    df = df_dict[batch_name]
    cell_line = cell_line_info[batch_name]  
    max_eff[cell_line] = max(max_eff[cell_line], df.differentiation_efficiency_index.max())

for batch_name in batch_names:
    df = df_dict[batch_name]
    cell_line = cell_line_info[batch_name]
    df.loc[:, "normalized_efficiency"] = df.differentiation_efficiency_index / max_eff[cell_line]
    df.to_pickle("./dataset/%s.pkl" % batch_name)
    df.to_csv("./dataset/%s.csv" % batch_name)

## Extract features

Then we extracted 343 features from the brightfield images and the cell region masks (which are also derived from the brightfield images). These features include:

* Area, Circumference, Area/Circumference Ratio, Solidity, Convexity, Circularity, Spacing, Max Centroid-Contour Distance (CCD), Min CCD, Min/Max Ratio of CCD, Mean of CCD, Standard Deviation of CCD : these feature were computed using the cell region masks.

* Total Variation, Hu Moment 1-7, SIFT 1-256, ORB 1-64: these features were computed using the brightfield images.

* Local Entropy, Cell Brightness, Cell Contrast: these features were computed using both the brightfield images and the cell region masks.


**Note**: SIFT 1-256 and ORB 1-64 are "bag-of-keywords" representation computed from local feature descriptors, which requires a training phase. We have trained the two feature extractor using bright-field images (saved at `./image_data/additional_images_for_training_local_feature_extractors/*.png`, which were excluded from the dataset) and cached them in `./utils/feature_extractor_sift.pkl` and `feature_extractor_orb.pkl`, so the following code can be run directly. 

If you want to reproduce the training of the SIFT and ORB feature extractor yourself, you can uncomment the code in the next block and run it; otherwise simply skip it.

In [ ]:
### You should uncomment the following code if you want to retrain the SIFT and ORB feature extractor.

# from utils.local_features import BagOfKeypointsRepresentation

# brightfield_imgs = glob.glob(os.path.join(image_data_dir, "additional_images_for_training_local_feature_extractors", "*.png"))

# orb = BagOfKeypointsRepresentation(
#     random_state = 123, 
#     candidates_for_each = 50, 
#     clusters = 64, 
#     des_type="ORB"
# )
# orb.fit(brightfield_imgs)
# orb.save("./utils/feature_extractor_orb.pkl")

# sift = BagOfKeypointsRepresentation(
#     random_state = 123, 
#     candidates_for_each = 50, 
#     clusters = 256, 
#     des_type="SIFT"
# )
# sift.fit(brightfield_imgs)
# sift.save("./utils/feature_extractor_sift.pkl")

In [ ]:
from utils.utils import (
    compute_area_circumference, 
    compute_CCD_features, 
    compute_cell_region_grayscale_feature,
    compute_Hu_moment, 
    compute_local_features, 
    compute_solidity_convexity, 
    compute_spacing, 
    compute_total_variation
)

func_brightfield = [
    compute_total_variation,
    compute_Hu_moment, 
    compute_local_features, 
]

func_mask = [
    compute_area_circumference, 
    compute_solidity_convexity, 
    compute_spacing, 
    compute_CCD_features
]

func_brightfield_mask = [
    compute_cell_region_grayscale_feature
]

In [ ]:
for batch_name in batch_names:
    
    print("Computing batch %s" % batch_name)
    
    df = pd.read_pickle("./dataset/%s.pkl" % batch_name)
    
    for i in tqdm.tqdm(df.index):
        
        brightfield_dir = os.path.join(image_data_dir, batch_name, "brightfield", "S%03d.png" % df.S_id[i])
        mask_dir = os.path.join(image_data_dir, batch_name, "cell_region", "S%03d.png" % df.S_id[i])
        
        for func in func_mask:
            features = func(mask_dir)
            for key, value in features.items():
                df.at[i, key] = value
                
        for func in func_brightfield_mask:
            features = func(brightfield_dir, mask_dir)
            for key, value in features.items():
                df.at[i, key] = value

        for func in func_brightfield:
            features = func(brightfield_dir)
            for key, value in features.items():
                df.at[i, key] = value

    df.to_pickle("./dataset/%s.pkl" % batch_name)
    df.to_csv("./dataset/%s.csv" % batch_name)

## Split the dataset

Finally we randomly split the dataset into a training set (70%) and a test set (30%).

In [ ]:
random.seed(123)
np.random.seed(123)

for batch_name in ["CD00-%d" % i for i in [1, 3, 4, 10, 2, 6, 5, 9, 8, 7]]:
    
    df = pd.read_pickle("./dataset/%s.pkl" % batch_name)

    train_idx, test_idx = train_test_split(range(len(df)), train_size=0.7)
    df.loc[train_idx, "train_or_test"] = "Train"
    df.loc[test_idx, "train_or_test"] = "Test"
    df = pd.concat([df.loc[train_idx, :], df.loc[test_idx, :]], axis = 0)
    
    df.to_pickle("./dataset/%s.pkl" % batch_name)
    df.to_csv("./dataset/%s.csv" % batch_name)